In [4]:
import pandas as pd
from filterpy.kalman import KalmanFilter
from filterpy.common import Q_discrete_white_noise
import numpy as np
import matplotlib.pyplot as plt 
import plotly.graph_objects as go
from plotly_resampler import FigureResampler, FigureWidgetResampler

In [5]:
df = pd.read_parquet("../data/static_models/pump4_power_siso.par")
df = df.set_index("time")
df = df.rolling(window=40, center=True).median().dropna()
df = df.resample("5s").mean().dropna()

In [6]:
df['pump4_speed'] = df['pump4_speed'].apply(lambda x: max(0, x))


In [8]:
train_df = df["2024-01-09 13:00:00":"2024-01-09 17:00:00"]
test_df = df["2024-01-10":"2024-02-16 12:00:00"]

In [9]:
from src.system_identification import ARXParametersIdentification
from src.preprocessing import Preprocessor
import pandas as pd
from sysidentpy.basis_function._basis_function import Polynomial

In [10]:
FEATURES = "pump4_speed"
TARGET = 'pump4_power'

sys_id = ARXParametersIdentification(FEATURES, 
                            TARGET, 
                            dataframe= train_df, 
                            model_type= 'siso')

In [11]:
parameters = {
    'estimator': 'ridge_regression',
    'ridge_param': 2e4,
    'order_selection': False,
    'n_terms': 4,
    'extended_least_squares': False,
    'ylag': 2,  
    'xlag': 1, 
    'basis_function': Polynomial(degree=1)
}
    
fitted_model, model_parameters = sys_id.identify_arx_parameters(x_train=train_df[FEATURES], 
                                                                y_train= train_df[TARGET],
                                                                **parameters)
    
model_parameters

/home/alqua/Git/edge_mpc/sys_identification/src/sysidentpy/narmax_base.py:1034: RuntimeWarning:

divide by zero encountered in divide

/home/alqua/Git/edge_mpc/sys_identification/src/sysidentpy/narmax_base.py:1034: RuntimeWarning:

invalid value encountered in divide



,Regressors,Parameters,ERR
0,x1(k-1),2.7095E-02,9.9814E-01
1,y(k-1),2.3590E-01,NAN
2,y(k-2),1.9355E-01,NAN
3,1,1.5144E-03,NAN


In [12]:
yhat  = sys_id.predict(x_test= test_df[FEATURES], y_test= test_df[TARGET], n_steps_ahead= 200)

#val_plots = sys_id.residuals_analysis(yhat, df[TARGET].values)
#print(model_parameters)

RRSE: 0.12553433770073041
MSE: 8.903332688157198


In [13]:
test_df["yhat"] = yhat

sys_id.plot_features_and_target(test_df, ['yhat', 'pump4_power'])

FigureWidgetResampler({
    'data': [{'name': '<b style="color:sandybrown">[R]</b> yhat <i style="color:#fc9944">~54m</i>',
              'showlegend': True,
              'type': 'scattergl',
              'uid': '3b93c5ee-a31b-45df-908d-27b66a05c008',
              'x': array([datetime.datetime(2024, 1, 10, 0, 0, tzinfo=<UTC>),
                          datetime.datetime(2024, 1, 10, 0, 27, tzinfo=<UTC>),
                          datetime.datetime(2024, 1, 10, 0, 33, 30, tzinfo=<UTC>), ...,
                          datetime.datetime(2024, 2, 16, 9, 38, 45, tzinfo=<UTC>),
                          datetime.datetime(2024, 2, 16, 10, 36, 50, tzinfo=<UTC>),
                          datetime.datetime(2024, 2, 16, 11, 26, 45, tzinfo=<UTC>)], dtype=object),
              'y': array([4.58600014e+01, 5.27372473e-02, 1.51441141e-03, ..., 2.65430542e-03,
                          1.51441141e-03, 2.65430542e-03])},
             {'name': ('<b style="color:sandybrown">[R' ... 'style="color:#fc9